In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation

Using Theano backend.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 99, in init_dev
    **args)
  File "pygpu/gpuarray.pyx", line 658, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 587, in pygpu.gpuarray.pygpu_init
pygpu.gpuarray.GpuArrayException: b'Could not load "libcuda.so": libnvidia-fatbinaryloader.so.384.130: cannot open shared object file: No such file or directory'


In [36]:
a=np.array([[0, 1, 2], [4, 6, 8], [9, 9 ,9]])

np.argmax(a[:,0])

2

In [57]:
def init_population(pop_size, dim, bounds=[-1,1]):
    '''
    This function initialize the population to be use in DE
    Arguments:
    pop_size - Number of individuals (there is no default value to this yet.).
    dim - dimension of the search space (default is 1).
    bounds - The inferior and superior limits respectively (default is [-1, 1]).
        '''
    return np.random.uniform(low=bounds[0], high=bounds[1], size=(pop_size, dim))

def keep_bounds(pop, bounds):
    '''
    This function keep the population in the seach space
    Arguments:
    pop - Population;
    bounds - The inferior and superior limits respectively
    '''
    pop[pop<bounds[0]] = bounds[0]; pop[pop>bounds[1]] = bounds[1]
    return pop

# Define the Fitness to be used in DE
def sp_fitness(target, score):
    '''
    Calculate the SP index and return the index of the best SP found
    
    Arguments:
    target: True labels
    score: the predicted labels
    '''
    from sklearn.metrics import roc_curve
    
    fpr, tpr, thresholds = roc_curve(target, score)
    jpr = 1. - fpr
    sp = np.sqrt( (tpr  + jpr)*.5 * np.sqrt(jpr*tpr) )
    idx = np.argmax(sp)
    return sp[idx], tpr[idx], fpr[idx]#sp, idx, sp[idx], tpr[idx], fpr[idx]

def set_weights_to_keras_model_and_compute_fitness(pop, data):
    '''
    This function will create a generic model and set the weights to this model and compute the fitness.
    Arguments:
    pop - The population of weights.
    data - The samples to be used to test.
    '''
    fitness = np.zeros((pop.shape[0],3))
    
    model = Sequential()
    model.add(Dense(5, input_dim=100, activation='tanh'))
    model.add(Dense(1, activation='tanh'))
    model.compile( loss='mean_squared_error', optimizer = 'rmsprop', metrics = ['accuracy'] )
    
    generic_weights = model.get_weights()
    hl_lim = generic_weights[0].shape[0]*generic_weights[0].shape[1]
    for ind in range(pop.shape[0]):
        w = []
        hl = pop[ind][:hl_lim]
        ol = pop[ind][hl_lim+generic_weights[1].shape[0]:hl_lim+generic_weights[1].shape[0]+generic_weights[1].shape[0]] 

        w.append(hl.reshape(generic_weights[0].shape))
        w.append(pop[ind][hl_lim:hl_lim+generic_weights[1].shape[0]])
        w.append(ol.reshape(generic_weights[2].shape))
        w.append(np.array(pop[ind][-1]).reshape(generic_weights[-1].shape))
        
        model.set_weights(w)
        y_score = model.predict(data[0])
        fitness[ind] = sp_fitness(target=data[1], score=y_score)
        #print('Population ind: {} - SP: {} - PD: {} - PF: {}'.format(ind, fitness[ind][0], fitness[ind][1], fitness[ind][2]))
    return fitness
        
def gen_rand(n_size=1):
    '''
    This function return a n_size-dimensional random vector.
    '''
    return np.random.random(n_size)

def evolution(pop, data, F, Cr, p, max_sp_evals):
    
    fitness = set_weights_to_keras_model_and_compute_fitness(pop, data)
    best_idx = np.argmax(fitness[:,0])
    print('Best NN found - SP: {} / PD: {} / FA: {}'.format(fitness[best_idx][0],
                                                            fitness[best_idx][1],
                                                            fitness[best_idx][2]))
    interactions = 0
    sp_evals = 0
    while sp_evals < max_sp_evals:
        print('===== Interaction: {} ====='.format(interactions))
        # ============ Mutation Step ===============
        mutant = np.zeros_like(pop)
        for ind in range(pop.shape[0]):
            tmp_pop = np.delete(pop, ind, axis=0)
            choices = np.random.choice(tmp_pop.shape[0], 1+2*p, replace=False)
            diffs = 0
            for idiff in range(1, len(choices), 2):
                diffs += F*((tmp_pop[choices[idiff]]-tmp_pop[choices[idiff+1]]))
                #mutant[ind] = tmp_pop[choices[0]] + diffs
                mutant[ind] = pop[best_idx] + diffs
        # keep the bounds
        mutant = keep_bounds(mutant, bounds=[-1,1])

        # ============ Crossover Step ============
        trial_pop = np.copy(pop)
        K = np.random.choice(trial_pop.shape[1])
        for ind in range(trial_pop.shape[0]):
            for jnd in range(trial_pop.shape[1]):
                if jnd == K or gen_rand()<Cr:
                    trial_pop[ind][jnd] = mutant[ind][jnd]
        # keep the bounds
        trial_pop = keep_bounds(trial_pop, bounds=[-1,1])
        
        trial_fitness = set_weights_to_keras_model_and_compute_fitness(trial_pop, data)
        sp_evals += pop.shape[0]
        
        winners = np.where(trial_fitness[:,0]>fitness[:,0])

        fitness[winners] = trial_fitness[winners]
        pop[winners] = trial_pop[winners]
        best_idx = np.argmax(fitness[:,0])
        print('Best NN found - SP: {} / PD: {} / FA: {}'.format(fitness[best_idx][0],
                                                                fitness[best_idx][1],
                                                                fitness[best_idx][2]))
        interactions += 1

In [3]:
data = np.load('/home/micael/MyWorkspace/RingerRepresentation/2channels/data17-18_13TeV.sgn_lhmedium_probes.EGAM2.bkg.vetolhvloose.EGAM7.samples.npz')
sgn = data['signalPatterns_etBin_2_etaBin_0']
bkg = data['backgroundPatterns_etBin_2_etaBin_0']

sgn_trgt = np.ones(sgn.shape[0])
bkg_trgt = -1*np.ones(bkg.shape[0])

sgn_normalized = np.zeros_like(sgn)
for ind in range(sgn.shape[0]):
    sgn_normalized[ind] = sgn[ind]/np.abs(np.sum(sgn[ind]))
    
bkg_normalized = np.zeros_like(bkg)
for ind in range(bkg.shape[0]):
    bkg_normalized[ind] = bkg[ind]/np.abs(np.sum(bkg[ind]))

dataset = np.append(sgn_normalized, bkg_normalized, axis=0)
trgt = np.append(sgn_trgt, bkg_trgt)

In [58]:
population = init_population(pop_size=100, dim=511, bounds=[-1,1])

In [60]:
max_sp = np.int(1e5)

In [61]:
evolution(pop=population, data=(dataset, trgt), F=0.8, Cr=0.9, p=1, max_sp_evals=max_sp)

Best NN found - SP: 0.8927220098394869 / PD: 0.9105609616485404 / FA: 0.12494045743347722
===== Interaction: 0 =====
Best NN found - SP: 0.8927220098394869 / PD: 0.9105609616485404 / FA: 0.12494045743347722
===== Interaction: 1 =====
Best NN found - SP: 0.8927220098394869 / PD: 0.9105609616485404 / FA: 0.12494045743347722
===== Interaction: 2 =====
Best NN found - SP: 0.9006733332668979 / PD: 0.9247281053234115 / FA: 0.12306440856826692
===== Interaction: 3 =====
Best NN found - SP: 0.9045477288886202 / PD: 0.9202919290211792 / FA: 0.11106062715727298
===== Interaction: 4 =====
Best NN found - SP: 0.907889617268282 / PD: 0.9450486548368632 / FA: 0.12852400389866406
===== Interaction: 5 =====
Best NN found - SP: 0.9150417981779051 / PD: 0.9446193474527762 / FA: 0.11406523666796134
===== Interaction: 6 =====
Best NN found - SP: 0.9187798530790767 / PD: 0.9386090440755581 / FA: 0.10083762650505287
===== Interaction: 7 =====
Best NN found - SP: 0.9187798530790767 / PD: 0.9386090440755581 /

KeyboardInterrupt: 

In [ ]:
model = Sequential()
model.add(Dense(5, input_dim=100, activation='tanh'))
model.add(Dense(1, activation='tanh'))


model.compile( loss='mean_squared_error', optimizer = 'rmsprop', metrics = ['accuracy'] )

model.summary()

In [ ]:
weights = model.get_weights()

In [ ]:
flat = weights[0].flatten()
print(flat.shape, weights[0].shape, weights[1].shape, weights[2].flatten().shape, weights[3].shape)

all_flat = np.concatenate([flat, weights[1], weights[2].flatten(), weights[3]])
all_flat.shape

In [ ]:
weights[2].shape

In [ ]:
a = weights[0].shape[0]*weights[0].shape[1]

In [ ]:
hl = all_flat[:a]
ol = all_flat[a+weights[1].shape[0]:a+weights[1].shape[0]+weights[1].shape[0]] 
print('Pesos Camada escondida: ',hl.reshape(weights[0].shape) == weights[0])
print('Bias Camada escondida: ', all_flat[a:a+weights[1].shape[0]] == weights[1])
print('Pesos Camada de saida: ', ol.reshape(weights[2].shape) == weights[2])
print('Bias Camada de saida: ', all_flat[-1] == weights[3])